# Pipeline de entrenamiento y predicción

En este notebook se ejemplifica como ejecutar los scripts para poder:
    
    1-Entrenar un modelo para clasificar las zonas de un mapa
    2-Usar ese mismo modelo para analizar zonas y tener un estimativo de las zonas 

## Preparando los datos

Para ejecutar este noteboook de prueba, podemos bajar imágenes raster de ejemplo ejecutando los siguientes bloques de código con los cuales descargaremos mapas de la ciudad de concordia para el año 2016 (que usaremos para entrenar el modelo) y 2018 (donde haremos la evaluación del mismo)

In [1]:
import os
root_path = '/ap-siu-habitat'
#root_path = os.getcwd()

# Traemos las imágenes de prueba
concordia_data_path = '{}/data/conae/centro/concordia'.format(root_path)
# Para entrenar usaremos el año 2016 de Concordia
train_product_folder = 'train'
full_train_folder = '{}/{}'.format(concordia_data_path,train_product_folder)
os.makedirs(full_train_folder, exist_ok=True)
train_product_image = 'concordia_2016.TIF'
train_tif_path = '{}/{}/{}'.format(concordia_data_path,train_product_folder,train_product_image)
train_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT7_PMS_201601281334024_ORT_C0000000058530_R1C1.TIF'

!wget -O $train_tif_path $train_remote_tif_url

# Para testear usaremos el año 2018 de Concordia
test_product_folder = 'test'
full_test_folder = '{}/{}'.format(concordia_data_path,test_product_folder)
os.makedirs(full_test_folder, exist_ok=True)
test_product_image = 'concordia_2018.TIF'
test_tif_path = '{}/{}/{}'.format(concordia_data_path,test_product_folder,test_product_image)
test_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT6_PMS_201801071323410_ORT_C0000000058510_R1C1.TIF'

!wget -O $test_tif_path $test_remote_tif_url



--2019-10-27 22:42:38--  https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT7_PMS_201601281334024_ORT_C0000000058530_R1C1.TIF
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 2607:f8b0:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 425903550 (406M) [image/tiff]
Saving to: '/ap-siu-habitat/data/conae/centro/concordia/train/concordia_2016.TIF'

/ap-siu-habitat/dat 100%[===================>] 406.17M   126MB/s    in 3.2s    

2019-10-27 22:42:42 (126 MB/s) - '/ap-siu-habitat/data/conae/centro/concordia/train/concordia_2016.TIF' saved [425903550/425903550]

--2019-10-27 22:42:42--  https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT6_PMS_201801071323410_ORT_C0000000058510_R1C1.TIF
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 2607:f8b0:400c:c06::80
Conne

Inicializamos el resto de las variables para ejecutar los scripts, entre ellas la cantidad de hilos paralelos (`n_jobs`) , arboles (`trees`) y profundidad de los mismos (`depth`). Además indicamos las rutas de los scripts, archivos vectoriales etiquetados y rutas de salida.

In [2]:

script_train_path = '{}/script/siu_train.py'.format(root_path)
script_test_path = '{}/script/siu_test.py'.format(root_path)

n_jobs = 4
trees = 107
depth = 1007

train_shp_path = '{}/data/etiquetado/centro/concordia2016_train.shp'.format(root_path)
out_train_model_path = '{}/data/concordia2016_train_rf_t{}_d{}'.format(root_path,trees,depth)
model_file_name = 'concordia2016_train_rf_t{trs}_d{dpt}_rf_t{trs}_d{dpt}.txt'.format(trs=trees,dpt=depth)

test_shp_path = '{}/data/etiquetado/centro/concordia2018_train.shp'.format(root_path)
out_test_results_path = '{}/data/concordia2018_train_rf_t{}_d{}'.format(root_path,trees,depth)

out_train_model_path_full = '{}/model_train_rf_t{}_d{}/{}'.format(out_train_model_path,trees,depth,model_file_name)
out_test_results_path_train = '{}/data/concordia2016_test_rf_t{}_d{}'.format(root_path,trees,depth)

## Entrenamiento del modelo

Entrenamos a modo de ejemplo sobre los datos de la ciudad de concordia del año 2016

In [3]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
#!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_train_rf.log 2>&1

In [4]:
print("Entrenamiento terminado")

Entrenamiento terminado


Podemos analizar la carpeta de salida donde encontramos el resultado de performance y el modelo obtenido

In [5]:
out_folder = '{}/model_train_rf_t{}_d{}/'.format(out_train_model_path,trees,depth)
!ls -lth $out_folder

total 13M
-rw-r--r-- 1 root root 1.4K Oct 19 23:22 concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt
-rw-r--r-- 1 root root  13M Oct 19 23:22 concordia2016_train_rf_t107_d1007_rf_t107_d1007.txt


Abriendo el archivo terminado en `_acc.txt` vemos la matriz de confusión y las medidas de performance de entrenamiento

In [6]:
performance_file = '{}{}'.format(out_folder,'concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
!cat $performance_file

Error Matrix

                Observed
                --------
                C     1   C     2   C     3   C     4   C     5   Total   User(%)
                -------   -------   -------   -------   -------   -----   -------
Predicted C001| (365)     26        1         2         0         394     92.64
          C002| 92        (866)     1         3         0         962     90.02
          C003| 3         4         (355)     6         0         368     96.47
          C004| 0         2         4         (432)     0         438     98.63
          C005| 0         0         0         0         (1258)    1258    100.00
         Total| 460       898       361       443       1258      (3420)
   Producer(%)| 79.35     96.44     98.34     97.52     100.00            (95.79%)

Samples: 3,420

Statistics

Overall Accuracy (%): 95.79
Kappa: 0.94
Hamming loss: 0.04

Class report

             precision    recall  f1-score   support

          1       0.93      0.79      0.85       460
     

## Predicción usando el modelo aprendido

Usamos el modelo aprendido para analizar la misma zona pero del año 2018


In [7]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
#!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path
#!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
# Sobre el test set (2018)
!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2018_test_rf.log 2>&1
# Sobre el train set (2016)
!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_test_train_rf.log 2>&1

In [8]:
print("Evaluacion terminada")

Evaluacion terminada


La performance sobre el año 2018 usado de test es la siguiente

In [9]:
out_folder = '{}/model_test_rf_t{}_d{}/'.format(out_test_results_path,trees,depth)
performance_file = '{}{}'.format(out_folder,'concordia2018_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
!cat $performance_file

Error Matrix

                Observed
                --------
                C     1   C     2   C     3   C     4   C     5   Total   User(%)
                -------   -------   -------   -------   -------   -----   -------
Predicted C001| (140)     452       36        73        0         701     19.97
          C002| 105       (330)     24        16        3         478     69.04
          C003| 9         1         (556)     4         45        615     90.41
          C004| 1         3         100       (158)     22        284     55.63
          C005| 0         0         0         0         (2264)    2264    100.00
         Total| 255       786       716       251       2334      (4342)
   Producer(%)| 54.90     41.98     77.65     62.95     97.00             (79.41%)

Samples: 4,342

Statistics

Overall Accuracy (%): 79.41
Kappa: 0.69
Hamming loss: 0.21

Class report

             precision    recall  f1-score   support

          1       0.20      0.55      0.29       255
     

Vemos que en la carpeta `clasificacion_...` dentro de la carpeta de salida, está la imagen obtenida luego de aplicar el modelo

In [10]:
map_file = '{}/clasificacion_test_rf_t{}_d{}/concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif'.format(out_test_results_path,trees,depth)
!ls $map_file

/ap-siu-habitat/data/concordia2018_train_rf_t107_d1007/clasificacion_test_rf_t107_d1007/concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif


## Mover los resultados a una carpeta con formato esperado para post-procesamiento

Movemos los resultados a una estructura de carpetas con el patrón `region/ciudad/año` que esperan los scripts de post-procesamiento 

In [11]:
# Script para tomar los resultados de clasificaciones para un año dado y generar la estructura de archivos necesaria para post-procesar y obtener estadísticas y plots
!script/prepare_results.sh\
    2016\
    centro/concordia\
    data/resultados/finales\
    data/concordia2016_train_rf_t107_d1007/clasificacion_train_rf_t107_d1007\
    data/conae/centro/concordia/SPOT7_20160128_1334024_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT7_001/VOL_SPOT7_001_A/IMG_SPOT7_PMS_001_A\
    data/imagenes_orig


0...10...20...30...40...50ERROR 4: data/conae/centro/concordia/SPOT7_20160128_1334024_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT7_001/VOL_SPOT7_001_A/IMG_SPOT7_PMS_001_A/*.tif: No such file or directory
Warning 1: Can't open data/conae/centro/concordia/SPOT7_20160128_1334024_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT7_001/VOL_SPOT7_001_A/IMG_SPOT7_PMS_001_A/*.tif. Skipping it
...60...70...80...90...100 - done.
ERROR 4: data/conae/centro/concordia/SPOT7_20160128_1334024_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT7_001/VOL_SPOT7_001_A/IMG_SPOT7_PMS_001_A/*.TIF: No such file or directory
Warning 1: Can't open data/conae/centro/concordia/SPOT7_20160128_1334024_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT7_001/VOL_SPOT7_001_A/IMG_SPOT7_PMS_001_A/*.TIF. Skipping it


In [12]:
!script/prepare_results.sh\
    2018\
    centro/concordia data/resultados/finales\
    data/concordia2018_train_rf_t107_d1007/clasificacion_test_rf_t107_d1007\
    data/conae/centro/concordia/SPOT6_20180107_1323410_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A\
    data/imagenes_orig



0...10...20...30...40...50ERROR 4: data/conae/centro/concordia/SPOT6_20180107_1323410_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/*.tif: No such file or directory
Warning 1: Can't open data/conae/centro/concordia/SPOT6_20180107_1323410_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/*.tif. Skipping it
...60...70...80...90...100 - done.
ERROR 4: data/conae/centro/concordia/SPOT6_20180107_1323410_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/*.TIF: No such file or directory
Warning 1: Can't open data/conae/centro/concordia/SPOT6_20180107_1323410_PMS_W058S31_L2A_16GT_010x011_4326/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_PMS_001_A/*.TIF. Skipping it
